Руководство решает внедрить фичу “С этим товаром покупают” в Интернет-магазине. Вам предлагается протестировать фичу на одном из товаров. Для тестирования фичи вам исходя из истории покупок в интернет-магазине нужно определить ТОП-10 наиболее близких товаров к исходному.

## Задание №1
Используйте датасет http://archive.ics.uci.edu/ml/datasets/online+retail

In [2]:
import pandas as pd
import numpy as np
df = pd.read_excel('http://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx')
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


## Задание №2
Создайте матрицу item-customer (по строкам - товары, по столбцам - покупатели)

In [5]:
df = df.loc[df['Quantity'] > 0]
df = df.dropna(subset=['CustomerID'])

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 397924 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    397924 non-null  object        
 1   StockCode    397924 non-null  object        
 2   Description  397924 non-null  object        
 3   Quantity     397924 non-null  int64         
 4   InvoiceDate  397924 non-null  datetime64[ns]
 5   UnitPrice    397924 non-null  float64       
 6   CustomerID   397924 non-null  float64       
 7   Country      397924 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 27.3+ MB


In [7]:
customer_item_matrix = df.pivot_table(index='StockCode', columns='CustomerID', values='Quantity', aggfunc='sum')
customer_item_matrix = customer_item_matrix.applymap(lambda x: 1 if x > 0 else 0)
customer_item_matrix.sample(n=21)

CustomerID,12346.0,12347.0,12348.0,12349.0,12350.0,12352.0,12353.0,12354.0,12355.0,12356.0,...,18273.0,18274.0,18276.0,18277.0,18278.0,18280.0,18281.0,18282.0,18283.0,18287.0
StockCode,,,,,,,,,,,,,,,,,,,,,
21888,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
22194,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
85059,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
90003B,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
35833P,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
23141,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
17136A,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
22233,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
84609,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Задание №3
Проведите оценку мер близости товаров, получив матрицу item_item_sim_matrix со значениями косинусов между векторами товаров

In [8]:
from scipy.sparse import csr_matrix
def cos_sim(array):
    csr_array = csr_matrix(array)
    numerator = np.dot(csr_array, csr_array.T).toarray()
    denominator = np.linalg.norm(array, axis=1, keepdims= True)*np.linalg.norm(array.T, axis=0, keepdims= True)
    return numerator/denominator
item_item_sim_matrix = cos_sim(customer_item_matrix.values)
item_item_sim_matrix

array([[1.        , 0.        , 0.09486833, ..., 0.06691496, 0.        ,
        0.07821652],
       [0.        , 1.        , 0.        , ..., 0.01618174, 0.        ,
        0.        ],
       [0.09486833, 0.        , 1.        , ..., 0.07053456, 0.        ,
        0.01099299],
       ...,
       [0.06691496, 0.01618174, 0.07053456, ..., 1.        , 0.        ,
        0.0775386 ],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.07821652, 0.        , 0.01099299, ..., 0.0775386 , 0.        ,
        1.        ]])

In [9]:
item_item_sim_matrix = pd.DataFrame(item_item_sim_matrix)
item_item_sim_matrix.columns = customer_item_matrix.index
item_item_sim_matrix['StockCode'] = customer_item_matrix.index
item_item_sim_matrix = item_item_sim_matrix.set_index('StockCode')
item_item_sim_matrix.head()

StockCode,10002,10080,10120,10125,10133,10135,11001,15030,15034,15036,...,90214V,90214W,90214Y,90214Z,BANK CHARGES,C2,DOT,M,PADS,POST
StockCode,,,,,,,,,,,,,,,,,,,,,
10002,1.000000,0.000000,0.094868,0.090351,0.062932,0.098907,0.095346,0.047673,0.075593,0.090815,...,0.0,0.0,0.0,0.0,0.0,0.029361,0.0,0.066915,0.000000,0.078217
10080,0.000000,1.000000,0.000000,0.032774,0.045655,0.047836,0.000000,0.000000,0.082261,0.049413,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.016182,0.000000,0.000000
10120,0.094868,0.000000,1.000000,0.057143,0.059702,0.041703,0.060302,0.060302,0.095618,0.028718,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.070535,0.000000,0.010993
10125,0.090351,0.032774,0.057143,1.000000,0.042644,0.044682,0.043073,0.000000,0.051224,0.030770,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.070535,0.000000,0.070669
10133,0.062932,0.045655,0.059702,0.042644,1.000000,0.280097,0.045002,0.060003,0.071358,0.057152,...,0.0,0.0,0.0,0.0,0.0,0.036955,0.0,0.070185,0.049752,0.021877


## Задание №4
Отберите ТОП-10 похожих товаров по StockCode

In [10]:
top_10 = item_item_sim_matrix.loc[10002].sort_values(ascending=False).head(11)
top_10

StockCode
10002     1.000000
90103     0.223607
21826     0.197642
90101     0.182574
90059F    0.182574
90059E    0.182574
90059C    0.182574
90059B    0.182574
16010     0.182574
84535A    0.181369
21439     0.177123
Name: 10002, dtype: float64

## Задание №5
Выведите список ТОП-10 похожих товаров с названиями (Description) на экран

In [44]:
df.loc[df['StockCode'].isin(top_10.index.tolist()), ['StockCode', 'Description']].drop_duplicates().set_index('StockCode')

,Description
StockCode,
10002,INFLATABLE POLITICAL GLOBE
90059B,DIAMANTE HAIR GRIP PACK/2 BLACK DIA
90059E,DIAMANTE HAIR GRIP PACK/2 RUBY
90059C,DIAMANTE HAIR GRIP PACK/2 MONTANA
21826,EIGHT PIECE DINOSAUR SET
21439,BASKET OF TOADSTOOLS
16010,FOLDING CAMPING SCISSOR W/KNIF & S
84535A,ENGLISH ROSE NOTEBOOK A6 SIZE
90059F,DIAMANTE HAIR GRIP PACK/2 LT ROSE
